## Run commands to show in browser:  
```
export JUPYTER_ALLOW_INSECURE_WRITES=1
jupyter notebook notebooks/data_visualize.ipynb

```

In [ ]:
import sys
sys.path.insert(0,'..')

import dtale
import pandas as pd

from configs import configs, paths
from steps import data_loader
from configs.parser import GeneralConfig, MultispectralConfig


In [ ]:
general_cfg = GeneralConfig(
	raster_num_closest_points=10,
	dates=["2022_06_15", "2022_07_11", "2022_07_20"],
	treatments = ["eko"],
	varieties=["KIS_Tamar", "KIS_Blegos"],
)
multispectral_cfg = MultispectralConfig(
	channels = ["blue", "green", "nir", "red_edge", "red"],
	location_type = "measured",
	rasters_paths = paths.PATHS_MULTISPECTRAL_IMAGES,
	shapefiles_paths = paths.PATHS_SHAPEFILES
)

In [ ]:
data = data_loader(general_cfg, multispectral_cfg)

In [ ]:
# merge two dataframes
df = pd.concat([data.data, data.meta], axis=1)
# Assigning a reference to a running D-Tale process.
d = dtale.show(df)
# Using Python's `webbrowser` package it will try and open your server's default browser to this process.
d.open_browser()

In [ ]:
# Shutting down D-Tale process
# d.kill()

In [ ]:
import sys
sys.path.insert(0,'..')

import pygwalker as pyg
import pandas as pd
from rich import print

from configs import paths, configs

In [ ]:
regression_label="LICOR_leaf"
excel_file = paths.PATHS_MEASUREMENTS[regression_label][1]
print(excel_file)


In [ ]:
df = pd.read_excel(excel_file)
walker = pyg.walk(
    df,
    spec=str(configs.SAVE_DIR / "walker_spec.json"),    # this json file will save your chart state, you need to click save button in ui mannual when you finish a chart, 'autosave' will be supported in the future.
    use_kernel_calc=True,          # set `use_kernel_calc=True`, pygwalker will use duckdb as computing engine, it support you explore bigger dataset(<=100GB).
    dark="light",
    store_chart_data=True,          
    hideDataSourceConfig=False
)